In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas_datareader.data as web
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.metrics import brier_score_loss, roc_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler

/Users/xiaozhezhang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


| market_category | feature_name | id |
|-----------------|--------------|----|
| Bank            | bac          |  1 |
| Bank            | citi         |  2 |
| Commodity       | corn         |  3 |
| Currency        | euro         |  4 |
| Commodity       | gold         |  5 |
| Inflation       | infl5y       |  6 |
| Commodity       | iyr          |  7 |
| Currency        | pound        |  8 |
| Commodity       | silver       |  9 |
| Commodity       | soybns       | 10 |
| Equity          | sp12m        | 11 |
| Equity          | sp6m         | 12 |
| Commodity       | wheat        | 13 |
| Currency        | yen          | 14 |


Return Model (Log Price)

In [2]:
df = pd.read_csv("mpd_sp500.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [3]:
# forwards filling
df = df.fillna(method='ffill')

In [4]:
# create a new df that extract the columns of SP_adj_close	SP_lg_pr	SP_lg_ret(%)	VIX
data = df[['SP_adj_close', 'SP_lg_pr', 'SP_lg_ret(%)', 'VIX']]
data

,SP_adj_close,SP_lg_pr,SP_lg_ret(%),VIX
Date,,,,
2013-01-17,1480.939941,7.300432,0.597345,0.1357
2013-01-24,1494.819946,7.309761,0.932878,0.1269
2013-01-31,1498.109985,7.311960,0.219854,0.1428
2013-02-07,1509.390015,7.319461,0.750130,0.1350
2013-02-14,1521.380005,7.327373,0.791222,0.1266
...,...,...,...,...
2024-01-10,4783.450195,8.472917,1.657668,0.1269
2024-01-17,4739.209961,8.463626,-0.929164,0.1479
2024-01-24,4868.549805,8.490551,2.692566,0.1314


In [5]:
# keep columns that have names containing f11 and f12 only
df = df.filter(regex='f11|f12')


In [6]:
# merge data to df merge on index
df = pd.merge(df, data, left_index=True, right_index=True, how='left')

In [7]:
# drop columns that has "maturity_target" , "lg_change_decr", and "lg_change_incr" in the column name; those are irrelevant for feature selection
df = df[df.columns.drop(list(df.filter(regex='maturity_target')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_decr')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_incr')))]
df = df[df.columns.drop(list(df.filter(regex='SP_adj_close')))]

# drop SP_lg_ret(%)	
df = df.drop(['SP_lg_ret(%)'], axis=1)
# df = df.drop(['SP_lg_pr'], axis=1)
df = df.drop(['VIX'], axis=1)



In [8]:
# Generate lagged variables from f1_mu to SP_lg_pr
lags = 6
for lag in range(1, lags+1):
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret_vol')+1]:
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('VIX')+1]: 
    for col in df.columns[df.columns.get_loc('f11_mu'):df.columns.get_loc('SP_lg_pr')+1]: 
    #for col in df.columns[df.columns.get_loc('f11_mu'):df.columns.get_loc('SP_lg_ret(%)')+1]:    
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_28857/2483780013.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_28857/2483780013.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_28857/2483780013.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

In [9]:
df_lagged = df.copy()
# drop NA rows
df_lagged = df_lagged.dropna()
df_lagged.shape


(571, 133)

In [10]:
df_lagged.head(20)

,f11_mu,f11_sd,f11_skew,f11_kurt,f11_p10,f11_p50,f11_p90,f11_prDec,f11_prInc,f12_mu,...,f12_mu_lag6,f12_sd_lag6,f12_skew_lag6,f12_kurt_lag6,f12_p10_lag6,f12_p50_lag6,f12_p90_lag6,f12_prDec_lag6,f12_prInc_lag6,SP_lg_pr_lag6
Date,,,,,,,,,,,,,,,,,,,,,
2013-02-28,-0.013760,0.165437,-1.064944,1.827118,-0.231609,0.014135,0.164019,0.123823,0.051886,-0.009874,...,-0.005964,0.112827,-1.083202,1.904765,-0.154080,0.013135,0.114733,0.063889,0.007340,7.300432
2013-03-07,-0.010876,0.156855,-1.125104,2.071839,-0.215251,0.015521,0.156367,0.111384,0.041900,-0.007606,...,-0.004666,0.101447,-1.053530,1.845878,-0.138036,0.012280,0.104440,0.049584,0.003538,7.309761
2013-03-14,-0.010876,0.156855,-1.125104,2.071839,-0.215251,0.015521,0.156367,0.111384,0.041900,-0.007606,...,-0.004666,0.101447,-1.053530,1.845878,-0.138036,0.012280,0.104440,0.049584,0.003538,7.311960
2013-03-21,-0.016487,0.168308,-1.137405,1.968257,-0.239122,0.013767,0.162133,0.128493,0.047310,-0.006612,...,-0.003163,0.101097,-1.121946,2.196441,-0.133458,0.013368,0.105116,0.048146,0.003592,7.319461
2013-03-28,-0.016487,0.168308,-1.137405,1.968257,-0.239122,0.013767,0.162133,0.128493,0.047310,-0.006612,...,-0.003163,0.101097,-1.121946,2.196441,-0.133458,0.013368,0.105116,0.048146,0.003592,7.327373
2013-04-04,-0.017964,0.170384,-1.143630,2.112269,-0.241237,0.011775,0.162035,0.129501,0.051591,-0.005958,...,-0.009874,0.112429,-1.038128,1.783160,-0.157649,0.008259,0.111115,0.065920,0.007199,7.314832
2013-04-11,-0.017964,0.170384,-1.143630,2.112269,-0.241237,0.011775,0.162035,0.129501,0.051591,-0.005958,...,-0.009874,0.112429,-1.038128,1.783160,-0.157649,0.008259,0.111115,0.065920,0.007199,7.322960
2013-04-18,-0.018118,0.176993,-1.053618,1.869310,-0.249189,0.010181,0.172501,0.135937,0.064454,-0.005473,...,-0.007606,0.105005,-1.098673,2.081670,-0.143297,0.009019,0.105231,0.055054,0.004169,7.342300
2013-04-25,-0.018118,0.176993,-1.053618,1.869310,-0.249189,0.010181,0.172501,0.135937,0.064454,-0.005473,...,-0.007606,0.105005,-1.098673,2.081670,-0.143297,0.009019,0.105231,0.055054,0.004169,7.354509


In [11]:
# df_lagged.to_csv('mpd_sp500_lagged_log_price.csv', index=False)

In [12]:
# Define the target variable
start_colunm = df_lagged.columns.get_loc('f11_mu_lag1')
# end_column = df_lagged.columns.get_loc('VIX_lag6')
end_column = df_lagged.columns.get_loc('SP_lg_pr_lag6')
#end_column = df_lagged.columns.get_loc('SP_lg_ret(%)_lag6')

column_index = list(range(start_colunm, end_column+1))

X = df_lagged.iloc[:, column_index]
# y = df_lagged['SP_lg_ret(%)'] 
y = df_lagged['SP_lg_pr'] 

split_index = int(len(X)*0.75)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [13]:
# X_train.head(20)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((428, 114), (143, 114), (428,), (143,))

In [15]:
# run a lasso regression to select features
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV.fit(X_train, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [16]:
print("In Sample R^2: ", f'{lassoCV.score(X_train, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV.score(X_test, y_test):.5f}')
print()
# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lass_y_pred = lassoCV.predict(X_test)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lass_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lass_y_pred)
lass_mape = np.mean(np.abs((y_test - lass_y_pred) / y_test)) * 100

print('Out of Sample Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')


In Sample R^2:  0.99314

Out of Sample R^2:  0.87785

Number of features selected:  12
                   coef
f11_kurt_lag1 -0.005614
f12_kurt_lag1  0.007945
SP_lg_pr_lag1  0.959054
f11_kurt_lag2  0.001958
SP_lg_pr_lag2  0.002003
f11_p10_lag4  -0.035641
f12_kurt_lag4 -0.005773
f11_kurt_lag5  0.007417
f12_kurt_lag5 -0.002573
f11_kurt_lag6 -0.004551
f12_kurt_lag6 -0.000052
SP_lg_pr_lag6  0.033627

Out of Sample Test set evaluation:
MSE: 0.00060, RMSE: 0.02458, MAE: 0.01869, MAPE: 0.22389


Applied StandardScaler

In [17]:
# run a lasso regression to select features
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
lassoCV2 = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV2.fit(X_train_scaled, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [19]:
print("In Sample R^2: ", f'{lassoCV2.score(X_train_scaled, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV2.score(X_test_scaled, y_test):.5f}')
print()

# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV2.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lassCV2_y_pred = lassoCV2.predict(X_test_scaled)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lassCV2_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lassCV2_y_pred)
lass_mape = np.mean(np.abs((y_test - lassCV2_y_pred) / y_test)) * 100

print('Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')


In Sample R^2:  0.99301

Out of Sample R^2:  0.87591

Number of features selected:  7
                    coef
f12_mu_lag1     0.000848
SP_lg_pr_lag1   0.213467
SP_lg_pr_lag2   0.017935
f12_p50_lag3    0.000948
f12_prInc_lag3  0.003012
f11_prInc_lag4  0.000592
f12_prInc_lag4  0.001080

Test set evaluation:
MSE: 0.00061, RMSE: 0.02478, MAE: 0.01900, MAPE: 0.22764


Using Log Price Lasso Regression, unscalered has better result (LassoCV)

Rolling Lasso Model

In [20]:
from sklearn.linear_model import LassoCV

# Initialize the LassoCV model
lassoCV_rolling = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')

# Initialize an empty array to store predictions
predictions = []

# initialize X_train_rolling and y_train_rolling
X_train_rolling = X_train.copy()
y_train_rolling = y_train.copy()

# Iterate through the dataset
for i in range(len(X_test)):
    # Convert X_train back to a DataFrame
    
    
    # Add the new observation to X_train_df and y_train
    X_train_rolling = pd.concat([X_train_rolling, X_test.iloc[[i]]])
    y_train_rolling = np.append(y_train_rolling, y_test[i])
    
    # Fit the LassoCV model with the updated training data
    lassoCV_rolling.fit(X_train_rolling, y_train_rolling)
    
    # Predict the next day's y
    next_day_prediction = lassoCV_rolling.predict(X_test.iloc[[i]])
    
    # Store the prediction in the array
    predictions.append(next_day_prediction)
    
    # Print or store the prediction for the next day
    # print(f"Day {i+1} Prediction: {next_day_prediction}")

In [21]:
predictions

[array([8.306013]),
 array([8.3238329]),
 array([8.34139833]),
 array([8.35053609]),
 array([8.34646105]),
 array([8.3476247]),
 array([8.3552422]),
 array([8.36495126]),
 array([8.38134448]),
 array([8.38282365]),
 array([8.38227696]),
 array([8.39274607]),
 array([8.39126406]),
 array([8.39501053]),
 array([8.38586268]),
 array([8.40672105]),
 array([8.41797845]),
 array([8.41659119]),
 array([8.40581247]),
 array([8.38915887]),
 array([8.37918392]),
 array([8.38012182]),
 array([8.38129619]),
 array([8.41981475]),
 array([8.42290977]),
 array([8.45068258]),
 array([8.44105975]),
 array([8.45214018]),
 array([8.4520785]),
 array([8.41967058]),
 array([8.45267612]),
 array([8.46146915]),
 array([8.46103093]),
 array([8.4731896]),
 array([8.4591416]),
 array([8.45994144]),
 array([8.41461679]),
 array([8.38346917]),
 array([8.43046733]),
 array([8.43547175]),
 array([8.40768688]),
 array([8.35027481]),
 array([8.38643094]),
 array([8.36075574]),
 array([8.38408948]),
 array([8.40163833

In [22]:
# make a copy of the predictions
Rolling_y_pred= np.array(predictions).flatten()


In [23]:
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, Rolling_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, Rolling_y_pred)
lass_mape = np.mean(np.abs((y_test - Rolling_y_pred) / y_test)) * 100

print('Out of Sample Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')

Out of Sample Test set evaluation:
MSE: 0.00057, RMSE: 0.02381, MAE: 0.01818, MAPE: 0.21783


In [24]:
# convert to dataframe
Rolling_y_pred = pd.DataFrame(lassCV2_y_pred)
# rename to Predicted_SP_lg_pr
Rolling_y_pred.columns = ['Predicted_SP_lg_pr']
Rolling_y_pred.to_csv('Rolling_Predicted_SP_lg_pr.csv', index=False)